# Querying Tables

### Introduction

Now so far in our journey through SQL we've worked with a single table.  We can say that we've focused on single table queries.  In this table we'll see what it's like to work with a project that has multiple tables, and how we can see the relationships between entities in different tables.

### Loading our data

In [3]:
import pandas as pd
albums_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/0-multi-tables/albums.csv"
albums_df = pd.read_csv(albums_url)
artists_url = "https://raw.githubusercontent.com/sql-fundamentals-jigsaw/mod-1-sql-curriculum/master/2-sql-relations/0-multi-tables/artists.csv"
artists_df = pd.read_csv(artists_url)

In [4]:
albums_df = pd.read_csv(albums_url, index_col = 0)
albums_df

,date,artist_id,tracks,track_length,title
id,,,,,
1,2019-01-19,2,17,256500.0,The Very Best Of Prince
2,2017-06-03,2,40,233901.0,4Ever
3,2016-07-16,1,11,202174.0,Stranger To Stranger


In [5]:
import sqlite3
conn = sqlite3.connect('billboard_sample.db')

In [6]:
albums_df.to_sql('albums', conn, index = False)
artists_df.to_sql('artists', conn, index = False)

2

In [7]:
cursor = conn.cursor()

In [8]:
pd.read_sql('select * from artists;', conn)

,id,name
0,1,Paul Simon
1,2,Prince


In [9]:
pd.read_sql('select * from albums;', conn)

,date,artist_id,tracks,track_length,title
0,2019-01-19,2,17,256500.0,The Very Best Of Prince
1,2017-06-03,2,40,233901.0,4Ever
2,2016-07-16,1,11,202174.0,Stranger To Stranger


### Describing Relationships

Finally, let's take a moment to describe the relationship between albums and artists.  

We say that
* an artist `has many` albums, and
* an album `has one` artist.  

`albums`

| id| date    | length| track_length | artist_id | title
| --|:-------:|-------| --------------| -------| ------|
144 | '2019-01-19'| 17 | 256500 | 1| The Very Best Of Prince
17155| '2017-06-03' | 40| 233901| 1|  '4Ever'
6269 | '2016-07-16'  | 11 | 202174.0 | 2| 'Stranger To Stranger'  

`artists`

| id        | name           |
| ------------- |:-------------:|
| 1               |Prince | 1|
| 2      | Paul Simon  |   3|




### Querying our tables

Now that we have an idea of how our tables fit together, we can think about constructing queryies for our tables.  There are two questions we can ask:

1. Given an album, what is the artist
2. Given an artist, what are all of the listed albums

Let's tackle these in turn.  Once again these are our tables.

`artists`

| id        | name           |
| ------------- |:-------------:|
| 1               |Prince |
| 2      | Paul Simon      |


`albums`

| id| date    | rank| length| track_length | artist_id | title
| --|:-------:| ----|-------| --------------| -------| ------|
144 | '2019-01-19'|'143' | 17 | 256500 | 1| The Very Best Of Prince
17155| '2017-06-03'| '154'| 40| 233901| 1|  '4Ever'
6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger'  

This is the first question we want to answer:

* Find the artist who wrote "Stranger to Stranger".  

Here is the query:

```sql
SELECT name FROM artists

JOIN albums on albums.artist_id = artists.id

WHERE title = 'Stranger to Stranger'
```

1. The Join statement

The key part is the middle `JOIN` component.
  
```sql
JOIN albums on albums.artist_id = artists.id
```



This links up the rows of the table, by combining each row where the album's table `artist_id` column equals the artist's tables `id` column.  By joining on these two columns, we can think of sql as creating a joined table that looks like the following:

| id| date    | rank| length| track_length | artist_id | album    | id        | name           |
| --|:-------:| ----|-------| --------------| -------| ------|------------- |---|
|144 | '2019-01-19'|'143' | 17 | 256500 | 1| The Very Best Of Prince | 1     |Prince |
|17155| '2017-06-03'| '154'| 40| 233901| 1|  '4Ever' | 1               |Prince |
|6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger'  | 2      | Paul Simon      |

So notice that in the chart above, we have the columns from both tables linked together, and they are linked by having each row's `artist_id` equal to the id from the table `artists`.

2. The Where Statement

```sql
WHERE album = 'Stranger to Stranger'
```

Now that JOIN statement has linked up the two tables, the WHERE statement can scope down the rows of the table.  Here our WHERE statement says we are only interested in the rows where the album column is 'Stranger to Stranger'.

| id| date    | rank| length| track_length | artist_id | album    | id        | name           |
| --|:-------:| ----|-------| --------------| -------| ------|------------- |---|
|6269 | '2016-07-16' | '68'| 11 | 202174.0 | 2| 'Stranger To Stranger'  | 2      | Paul Simon      |

3. The SELECT statement

```SQL
SELECT name FROM artists
```

Finally, the SELECT statement indicates the columns that should be returned from the selected rows.  Here, it's the name column.

name           |
---|
 Paul Simon      |

Which answers our question:
    
* Find the artist who wrote "Stranger to Stranger".  

And which came from the query:

```sql
SELECT name FROM artists
JOIN albums on albums.artist_id = artists.id
WHERE album = 'Stranger to Stranger'
```

### Table Prefixes

Now that we are working with more than one table, it's a good time to introduce the idea of table prefixes.  In the query above, where we say `artists.id` we are using a table prefix.  The table name is `artists` and the column on that table is `id`.  So `artists.id` allows us to specify a column by first indicating the table the column is on.  We do so with the pattern: `table_name.column`.

You'll often see the table prefixes in the select statement.  For example, we could have written the above statement as:

```sql
SELECT artists.name FROM artists
JOIN albums on albums.artist_id = artists.id
WHERE album = 'Stranger to Stranger'
```

Or if we wanted to return all of the columns on the artists table, we could have written the following:

```sql
SELECT artists.* FROM artists
JOIN albums on albums.artist_id = artists.id
WHERE album = 'Stranger to Stranger'
```

### Summary

In this lesson, we learned about how to

### Additional Resources

[music story api](http://developers.music-story.com/developers/artist)